<a href="https://colab.research.google.com/github/TynK-M/nursing-fall-risk-assessment/blob/main/nursing_fall_risk_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
low_ambulatory_aid = [
    "bed rest",
    "nurse assist",
]

medium_ambulatory_aid = [
    "crutches",
    "cane",
    "walker",
]

high_ambulatory_aid = [
    "furniture",
]

In [12]:
low_gait_or_transfering = [
    "normal",
    "bed rest",
    "immobile",
]

medium_gait_or_transfering = [
    "weak",
]

high_gait_or_transfering = [
    "impaired",
]

In [13]:
def mfr(falling_history: bool, secondary_diagnosis: bool, ambulatory_aid: str, iv_or_heparin_lock: bool, gait_or_transfering: str, does_forget_limitations: bool) -> int:
    score = 0

    score = score + 25 if falling_history else score
    score = score + 15 if secondary_diagnosis else score

    if ambulatory_aid in high_ambulatory_aid:
        score += 30
    elif ambulatory_aid in medium_ambulatory_aid:
        score += 15

    score = score + 20 if iv_or_heparin_lock else score

    if gait_or_transfering in high_gait_or_transfering:
        score += 20
    elif gait_or_transfering in medium_gait_or_transfering:
        score += 10

    score = score + 15 if does_forget_limitations else score

    return score